In [40]:
import os
import glob
import rasterio
import fiona

import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt

from osgeo import gdal
from shapely.geometry import Point

In [5]:
def read_file(filename):
    with open(filename) as f:
        contents = f.readlines()

    data = []
    for line in contents:
        if line[0:1] == "#":
            continue
        else:
            data.append(line)

    headers = [x.replace("Soil Moisture Percent","smp").replace(" ","_") for x in data[0].split(",")]
    cols = [x.strip("\n").split(",") for x in data[1:]]

    df = pd.DataFrame(cols, columns = headers)
    
    return df

def get_site_lat_lons(site_num, stations_csv):
    df = pd.read_csv(stations_csv)
    site = df[df['site_name'].str.contains(site_no)]
    
    lat, lon = site['lat'].values, site['lon'].values
    return lat[0], lon[0]

In [34]:
csvs = glob.glob("*.csv")
txts = glob.glob("*.txt")
df = pd.read_csv(csvs[0])
lats = df.lat
longs = df.lon

In [35]:
lats = df.lat
longs = df.lon

In [36]:
crs = {"init":"epsg:4326"}
geom = [Point(xy) for xy in zip(longs,lats)]

In [37]:
gdf = gp.GeoDataFrame(df, crs = crs, geometry = geom)

In [38]:
gdf.to_file("../shape/scan_sites.shp")

In [55]:
# Extract raster values at points:
clays = []
silts = []
sands = []

# Read input shapefile with fiona and iterate over each feature
with fiona.open('../shape/scan_sites.shp') as shp:
    for feature in shp:
        station_name = feature['properties']['site_name']
        coords = feature['geometry']['coordinates']
        # Read pixel value at the given coordinates using Rasterio
        # NB: `sample()` returns an iterable of ndarrays.
        with rasterio.open('../rasters/Unified_NA_Soil_Map_Subsoil_Clay_Fraction.tif') as src:
            clay = [v for v in src.sample([coords])][0][0]
        with rasterio.open('../rasters/Unified_NA_Soil_Map_Subsoil_Silt_Fraction.tif') as src:
            silt = [v for v in src.sample([coords])][0][0]
        with rasterio.open('../rasters/Unified_NA_Soil_Map_Subsoil_Sand_Fraction.tif') as src:
            sand = [v for v in src.sample([coords])][0][0]
        # Update the lists accordingly
        clays.append(clay)
        sands.append(sand)
        silts.append(silt)


In [56]:
df['clay_fract'] = clays
df['silt_fract'] = silts
df['sand_fract'] = sands

In [58]:
df.to_csv("../data/sites_textures.csv")

In [59]:
gdf['clay_fract'] = clays
gdf['silt_fract'] = silts
gdf['sand_fract'] = sands

In [60]:
gdf.to_file("../shape/scan_sites.shp")

In [61]:
gdf

,ntwk,state,site_name,ts,start,lat,lon,elev,county,huc,geometry,clay_fract,silt_fract,sand_fract
0,SCAN,AK,Aniak (2065),NaN,2002-July,61.58,-159.58,80,Bethel,Aniak Slough-Kuskokwim River (190305020504),POINT (-159.58 61.58),8.0,58.0,34.0
1,SCAN,AK,Canyon Lake (1232),NaN,2014-August,59.42,-161.16,550,Bethel,Canyon Creek (190305028402),POINT (-161.16 59.42),3.0,47.0,50.0
2,SCAN,AK,Checkers Creek (2213),NaN,2014-September,65.40,-164.71,326,Nome,Quartz Creek (190501050503),POINT (-164.71 65.40000000000001),5.0,58.0,37.0
3,SCAN,AK,Hozatka Lake (2210),NaN,2014-August,65.20,-156.63,206,Yukon-koyukuk,Hozatka Lake (190901091201),POINT (-156.63 65.2),8.0,58.0,34.0
4,SCAN,AK,Innoko Camp (2211),NaN,2014-August,63.64,-158.03,83,Yukon-koyukuk,190903031103-Innoko River (190903031103),POINT (-158.03 63.64),8.0,58.0,34.0
5,SCAN,AK,Kanaryagak Camp (2208),NaN,2013-August,61.36,-165.12,13,Kusilvak,Kuyungslik River (190903053903),POINT (-165.12 61.36),0.0,0.0,0.0
6,SCAN,AK,Kanuti Lake (2212),NaN,2014-August,66.18,-151.74,524,Yukon-koyukuk,Old Dummy Lake-Kanuti River (190901040707),POINT (-151.74 66.18000000000001),8.0,58.0,34.0
7,SCAN,AK,Lower Mulchatna (1233),NaN,2012-September,59.82,-156.99,320,Dillingham,190303021706-Mulchatna River (190303021706),POINT (-156.99 59.82),8.0,58.0,34.0
8,SCAN,AK,Moose Inc (2062),NaN,2002-May,59.68,-151.37,132,Kenai Cook Inlet,Fritz Creek (190203010805),POINT (-151.37 59.68),2.0,80.0,18.0
9,SCAN,AK,Naknek River (2209),NaN,2013-October,58.67,-156.57,100,Bristol Bay,Town of King Salmon-Naknek River (190302041502),POINT (-156.57 58.67),-999.0,-999.0,-999.0
